# SciSweeper class

In [1]:
from scisweeper import SciSweeper

# User defined class

In [2]:
ssw = SciSweeper(working_directory='.')

In [3]:
class BashSciSweeper(ssw.job):
    @property
    def executable(self): 
        return 'bash ~/tests/executable/test.sh'
    
    @staticmethod
    def write_input(input_dict, working_directory='.'): 
        import os
        from jinja2 import Template
        template = Template("{{value_1}} {{value_2}} {{value_3}}")
        template_str = template.render(value_1=input_dict['value_1'],
                                       value_2=input_dict['value_2'],
                                       value_3=input_dict['value_3'])
        with open(os.path.join(working_directory, 'input_file'), 'w') as f:
            f.writelines(template_str)

    @staticmethod
    def collect_output(working_directory='.'):
        import os
        with open(os.path.join(working_directory, 'output.log'), 'r') as f:
            output = f.readlines()
        return {'result': int(output[-1])}

In [4]:
ssw.job_class = BashSciSweeper

# Iterate over a given parameter range 

In [5]:
input_lst = []
for i in range(3):
    for j in range(3):
        for k in range(3):
            input_lst.append({'value_1': i, 'value_2': j, 'value_3': k})

# Custom job name - optional

In [6]:
def job_name(input_dict, counter):
    return 'job_' + str(counter) + '_' + str(input_dict['value_1'])

In [7]:
ssw.job_name_function = job_name

# Execute jobs in parallel

In [8]:
ssw.run_jobs_in_parallel(input_dict_lst=input_lst, cores=2)

# Collect results 

In [9]:
ssw.collect()

In [10]:
ssw.results

,dir,value_1,value_2,value_3,result
0,job_24_2,2,2,0,1
1,job_17_1,1,2,2,1
2,job_14_1,1,1,2,1
3,job_7_0,0,2,1,1
4,job_4_0,0,1,1,1
5,job_1_0,0,0,1,1
6,job_0_0,0,0,0,1
7,job_25_2,2,2,1,1
8,job_10_1,1,0,1,1
9,job_20_2,2,0,2,1


# Update output parser function - without running the simulation again

In [11]:
class BashSciSweeper2(ssw.job):
    @staticmethod
    def collect_output(working_directory='.'):
        import os
        with open(os.path.join(working_directory, 'output.log'), 'r') as f:
            output = f.readlines()
        return {'result': int(output[0])}

In [12]:
ssw.job_class = BashSciSweeper2

In [13]:
ssw.run_collect_output()

In [14]:
ssw.results

,dir,value_1,value_2,value_3,result
0,job_24_2,2,2,0,2
1,job_17_1,1,2,2,5
2,job_14_1,1,1,2,3
3,job_7_0,0,2,1,2
4,job_4_0,0,1,1,1
5,job_1_0,0,0,1,0
6,job_0_0,0,0,0,0
7,job_25_2,2,2,1,4
8,job_10_1,1,0,1,1
9,job_20_2,2,0,2,2


# Identify broken calculation

In [15]:
class BashSciSweeper3(ssw.job):
    @staticmethod
    def collect_output(working_directory='.'):
        import os
        with open(os.path.join(working_directory, 'output.log'), 'r') as f:
            output = f.readlines()
        if int(output[0]) == 1:
            return {}
        else:
            return {'result': int(output[0])}

In [16]:
ssw.job_class = BashSciSweeper3

In [17]:
ssw.run_collect_output()

In [18]:
ssw.collect()

In [19]:
ssw.results

,dir,value_1,value_2,value_3,result
0,job_24_2,2,2,0,2.0
1,job_17_1,1,2,2,5.0
2,job_14_1,1,1,2,3.0
3,job_7_0,0,2,1,2.0
4,job_4_0,0,1,1,NaN
5,job_1_0,0,0,1,0.0
6,job_0_0,0,0,0,0.0
7,job_25_2,2,2,1,4.0
8,job_10_1,1,0,1,NaN
9,job_20_2,2,0,2,2.0


In [20]:
ssw.broken_jobs

['/home/jovyan/notebooks/job_4_0',
 '/home/jovyan/notebooks/job_10_1',
 '/home/jovyan/notebooks/job_9_1',
 '/home/jovyan/notebooks/job_15_1',
 '/home/jovyan/notebooks/job_12_1',
 '/home/jovyan/notebooks/job_11_1']